In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import IPython.display as display
from itertools import product
from torch.utils.data import TensorDataset, DataLoader

class NeuralNetworkLayer(nn.Module):
    def __init__(self, hidden_size:int):
        super(NeuralNetworkLayer, self).__init__()

        #Layer 1 Input: 17 Output: 3
        self.layer1 = nn.Linear(17, hidden_size)
        #nn.init.kaiming_normal_(self.layer1.weight, mode='fan_in', nonlinearity='relu')
        nn.init.uniform_(self.layer1.weight, -0.7, 0.7)
        nn.init.constant_(self.layer1.bias, 0.01)
        #Layer 3 Input: 3 Output: 1
        self.layer2 = nn.Linear(hidden_size, 1)
        #nn.init.kaiming_normal_(self.layer2.weight, mode='fan_in', nonlinearity='relu')
        nn.init.uniform_(self.layer2.weight, -0.7, 0.7)
        nn.init.constant_(self.layer2.bias, 0.01)

    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        return x

In [2]:

def importMonkDataset(file_name:str) -> pd.DataFrame:
    dataset = None
    columns_name = ["Y"] + [f"X{i}" for i in range(1,7)] + ["ID"]
    try:
        dataset = pd.read_csv(file_name, sep=" ", names=columns_name)
    except Exception as e:
        print("Error | Parsing target dataset for validation!")
        print(e)
    dataset.set_index('ID', inplace=True)
    return dataset

def takeMonkInputDataset(dataset:pd.DataFrame) -> pd.DataFrame:
    return dataset.iloc[:, 1:] #Return dataset without first and last column
 
def takeMonkOutputDataset(dataset:pd.DataFrame) -> pd.DataFrame:
    return dataset.iloc[:,[0]] #Return dataset with only first column


def convert_x(x_train: np.ndarray):
    dict_3 = {1: [1, 0, 0], 2: [0, 1, 0], 3: [0, 0, 1]}
    dict_2 = {1: [1, 0], 2: [0, 1]}
    dict_4 = {1: [1, 0, 0, 0], 2: [0, 1, 0, 0], 3: [0, 0, 1, 0], 4: [0, 0, 0, 1]}

    new_y = []

    for row in x_train:
        new_row = []
        for j, value in enumerate(row):
            if j in [0, 1, 3]:
                new_row.extend(dict_3.get(value))
            elif j in [2, 5]:
                new_row.extend(dict_2.get(value))
            elif j == 4:
                new_row.extend(dict_4.get(value))

        new_y.append(new_row)
    return new_y

In [ ]:
dataset_train = importMonkDataset("MONK/monks-1.train")
x_train = takeMonkInputDataset(dataset_train)
y_train = takeMonkOutputDataset(dataset_train)

#Convert data
x_train = convert_x(x_train.to_numpy())
y_train = y_train.to_numpy()

x_train = torch.Tensor(x_train)
y_train = torch.Tensor(y_train)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

#Move data to gpu
x_train = x_train.to("cuda:0")
y_train = y_train.to("cuda:0")

x_train = x_train.double()
y_train = y_train.double()

dataset_train = TensorDataset(x_train, y_train)
data_loader = DataLoader(dataset_train, batch_size=10, shuffle=True)

In [ ]:
for p in net.parameters():
    print(p)

In [ ]:
# activation: tanh, relu
# solver: sgd, adam
# lr: 0.01, 0.001, 0.05, 0.005
# iter: 350, 400
# alpha: 0.0001, 0.001, 0.0005, 0.005
# layers: 2, 3, 4

# parameters = {'activation':('tanh', 'relu'), 
#               'solver':('sgd', 'adam'), 
#               'lr':('0.01', '0.001', '0.05', '0.005'), 
#               'epochs': ('350', '400', '450'),
#               'alpha' : ('0.0001', '0.001', '0.0005', '0.005'),
#               'layers' : ('2', '3', '4')}

#parameters = {'lr':('0.01', '0.001', '0.05', '0.005')}

In [ ]:
#Net
net = NeuralNetworkLayer(3)
#
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

loss_values = []
epochs = 390
for epoch in range(epochs):
    total_loss = 0
    for batch_input, batch_output in data_loader:
        #Forward pass
        outputs = net(batch_input)
        #Training loss
        loss = criterion(outputs, batch_output)
        #Calculate accuracy
        
        #Outputs convert to binary 0 or 1 with P (Probability)
        #outputsConv = (F.sigmoid(outputs) >= 0.5).float()
        #_, predicted = torch.max(outputsConv.data, 1)
        total_loss += loss.item()
        #Backward and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    avg_loss = total_loss / len(data_loader)
    #Add to list
    loss_values.append(avg_loss)

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

#Update plot
display.clear_output(wait=True)
plt.plot(loss_values, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss per Epoch')
plt.legend()
plt.show()

In [ ]:
loss_values = []
epochs = 390
for epoch in range(epochs):
    #Forward pass
    outputs = net(x_train)
    #
    outputs = (outputs > 0.5).float()
    #Training loss
    loss = criterion(outputs, y_train)
    #Backward and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #Add to list
    loss_values.append(loss.item())

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

#Update plot
display.clear_output(wait=True)
plt.plot(loss_values, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss per Epoch')
plt.legend()
plt.show()

In [ ]:
#torch.save(net, 'model390.pth')
#outputsCl = outputs.clamp(0,1)
#outputsB = (outputs > 0.5).float()

In [ ]:
dataset_test = importMonkDataset("MONK/monks-1.test")

x_test = takeMonkInputDataset(dataset_test)
y_test = takeMonkOutputDataset(dataset_test)
x_test = x_test.to_numpy()

x_test = convert_x(x_test)
y_test = y_test.to_numpy()

x_test = torch.tensor(x_test)
y_test = torch.tensor(y_test)

x_test = x_test.to("cuda:0")
y_test = y_test.to("cuda:0")

x_test = x_test.double()
y_test = y_test.double()

dataset_test = TensorDataset(x_test, y_test)
data_loader_test = DataLoader(dataset_test, batch_size=10, shuffle=True)
    

In [ ]:
from sklearn.model_selection import train_test_split

results = []
hidden_sizes = [2,3,4,5]
learning_rates = [0.01, 0.001, 0.05, 0.005]
rates_split = [0.2, 0.25, 0.3]

for hidden_size, learning_rate, rates in product(hidden_sizes, learning_rates, rates_split):

    train_data, test_data, train_target, test_target = train_test_split()

    net = NeuralNetworkLayer(hidden_size)
    net = net.to("cuda:0")
    net = net.double()

    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate)

    loss_values = []
    epochs = 390
    for epoch in range(epochs):
        total_loss = 0
        for batch_input, batch_output in data_loader:
            #Forward pass
            outputs = net(batch_input)
            #Training loss
            loss = criterion(outputs, batch_output)
            #Calculate total loss
            total_loss += loss.item()
            #Backward and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        avg_loss = total_loss / len(data_loader)
        #Add to list
        loss_values.append(avg_loss)

    #Save plot
    display.clear_output(wait=True)
    plt.plot(loss_values, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss per Epoch')
    plt.legend()
    plt.savefig(f'GridPlot/{hidden_size}-{learning_rate}.png')
    plt.clf()
    
    total = 0
    correct = 0
    net.eval()
    with torch.no_grad():
        for batch_input, batch_output in data_loader_test:
            outputs = net(batch_input)
            predicted = (outputs > 0.5).float()
            total += batch_input.size(0)
            correct += (predicted == batch_output).sum().item()

    accuracy = correct / total
    result = f'Hidden_size:{hidden_size}, Learning-rate:{learning_rate}, Accuracy: {accuracy:.4f}'
    print(result)
    results.append(result)

with open("results.txt", 'w') as file:
    for result in results:
        file.write(result + "\n")


In [ ]:
net.eval()
with torch.no_grad():
    outputs = net(x_test)
    predicted = (outputs > 0.5).float()
    correct = (predicted == y_test).sum().item()
    total = x_test.size(0)

accuracy = correct / total
print(f'Accuracy: {accuracy:.4f}')

In [3]:
from sklearn.model_selection import train_test_split
import os

results = []
hidden_sizes = [2,3,4,5]
learning_rates = [0.01, 0.001, 0.05, 0.005]
rates_split = [0.2, 0.25, 0.3]

## TRAIN
dataset_train = importMonkDataset("MONK/monks-1.train")
x_train = takeMonkInputDataset(dataset_train)
y_train = takeMonkOutputDataset(dataset_train)

#Convert data
x_train = convert_x(x_train.to_numpy())
y_train = y_train.to_numpy()

#x_train = torch.Tensor(x_train)
#y_train = torch.Tensor(y_train)

## TEST
dataset_val = importMonkDataset("MONK/monks-1.test")

x_val = takeMonkInputDataset(dataset_val)
y_val = takeMonkOutputDataset(dataset_val)
x_val = x_val.to_numpy()

x_val = convert_x(x_val)
y_val = y_val.to_numpy()

x_val = torch.tensor(x_val)
y_val = torch.tensor(y_val)

x_val = x_val.to("cuda:0")
y_val = y_val.to("cuda:0")

x_val = x_val.double()
y_val = y_val.double()

dataset_val = TensorDataset(x_val, y_val)
data_loader_val = DataLoader(dataset_val, batch_size=10, shuffle=True)





for hidden_size, learning_rate, rate in product(hidden_sizes, learning_rates, rates_split):

    train_data, test_data, train_target, test_target = train_test_split(x_train, y_train, test_size=rate, random_state=40)

    train_data, test_data, train_target, test_target = torch.tensor(train_data), torch.tensor(test_data), torch.tensor(train_target), torch.tensor(test_target)

    train_data, test_data, train_target, test_target = train_data.to("cuda:0"), test_data.to("cuda:0"), train_target.to("cuda:0"), test_target.to("cuda:0")

    train_data, test_data, train_target, test_target = train_data.double(), test_data.double(), train_target.double(), test_target.double()

    dataset_train = TensorDataset(train_data, train_target)
    data_loader_train = DataLoader(dataset_train, batch_size=10, shuffle=True)

    dataset_test = TensorDataset(test_data, test_target)
    data_loader_test = DataLoader(dataset_test, batch_size=10, shuffle=True)

    net = NeuralNetworkLayer(hidden_size)
    net = net.to("cuda:0")
    net = net.double()

    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate)

    loss_values = []
    accuracy_values = []
    epochs = 390
    for epoch in range(epochs):
        total_loss = 0
        for batch_input, batch_output in data_loader_train:
            #Forward pass
            outputs = net(batch_input)
            #Training loss
            loss = criterion(outputs, batch_output)
            #Calculate total loss
            total_loss += loss.item()
            #Backward and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        avg_loss = total_loss / len(data_loader_train)
        #Add to list
        loss_values.append(avg_loss)

        total = 0
        correct = 0
        net.eval()
        with torch.no_grad():
            for batch_input, batch_output in data_loader_test:
                outputs = net(batch_input)
                predicted = (outputs > 0.5).float()
                total += batch_input.size(0)
                correct += (predicted == batch_output).sum().item()
            accuracy = correct / total
            accuracy_values.append(accuracy)

        net.train()

    #Create dir
    path_name = f'GridPlot/test-{hidden_size}-{learning_rate}-{rate}'
    os.mkdir(path_name)
    #Save plot loss
    display.clear_output(wait=True)
    plt.plot(loss_values, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss per Epoch')
    plt.legend()
    plt.savefig(f'{path_name}/Loss.png')
    plt.clf()
    #Save plot accuracy
    display.clear_output(wait=True)
    plt.plot(accuracy_values, label='Accuracy Test')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy for Epoch')
    plt.legend()
    plt.savefig(f'{path_name}/Accuracy-test.png')
    plt.clf()
    
    total = 0
    correct = 0
    net.eval()
    with torch.no_grad():
        for batch_input, batch_output in data_loader_val:
            outputs = net(batch_input)
            predicted = (outputs > 0.5).float()
            total += batch_input.size(0)
            correct += (predicted == batch_output).sum().item()

    accuracy = correct / total
    result = f'Hidden_size:{hidden_size}, Learning-rate:{learning_rate}, Rate: {rate} ,Accuracy: {accuracy:.4f}'
    print(result)
    results.append(result)

with open("results.txt", 'w') as file:
    for result in results:
        file.write(result + "\n")


Hidden_size:5, Learning-rate:0.005, Rate: 0.3 ,Accuracy: 0.6852


<Figure size 640x480 with 0 Axes>